In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriatic_Arthritis/GSE141934'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional data of inflamatory arthritis T cells."
!Series_summary	"With a focus on rheumatoid arthritis (RA), we sought new insight into genetic mechanisms of adaptive immune dysregulation to help prioritise molecular pathways for targeting in this and related immune pathologies. Whole genome methylation and transcriptional data from isolated CD4+ T cells and B cells of >100 genotyped and phenotyped inflammatory arthritis patients, all of whom were naïve to immunomodulatory treatments, were obtained. Analysis integrated these comprehensive data with GWAS findings across IMDs and other publically available resources."
!Series_overall_design	"Suspected inflammatory arthritis patients of Northern European ancestry were recruited prior to treatment with immunomodulatory drugs. RA patients were classified using current, internationally accepted criteria, and matched with disease controls in respect of demographic and clinical characteristics. CD4

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True 

# Determine the keys for each variable
trait_row = 6  # Based on 'working_diagnosis' which includes 'Psoriatic Arthritis'
age_row = 2  # Based on 'age' data 
gender_row = 1  # Based on 'gender' data (F, M)

# Define conversion functions
def convert_trait(value):
    """
    Convert the trait value to binary.
    - Psoriatic Arthritis -> 1
    - Others -> 0
    """
    try:
        raw_value = value.split(':')[1].strip()
        if raw_value == 'Psoriatic Arthritis':
            return 1
        else:
            return 0
    except:
        return None

def convert_age(value):
    """
    Convert the age value to continuous.
    """
    try:
        raw_value = value.split(':')[1].strip()
        return float(raw_value)
    except:
        return None

def convert_gender(value):
    """
    Convert the gender value to binary.
    - Female (F) -> 0
    - Male (M) -> 1
    """
    try:
        raw_value = value.split(':')[1].strip()
        if raw_value == 'F':
            return 0
        elif raw_value == 'M':
            return 1
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE141934', './preprocessed/Psoriatic_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriatic_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriatic_Arthritis/trait_data/GSE141934.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4216498': [0.0, 50.0, 0.0], 'GSM4216499': [1.0, 43.0, 0.0], 'GSM4216500': [0.0, 66.0, 0.0], 'GSM4216501': [0.0, 55.0, 1.0], 'GSM4216502': [1.0, 52.0, 0.0], 'GSM4216503': [0.0, 54.0, 0.0], 'GSM4216504': [0.0, 63.0, 0.0], 'GSM4216505': [0.0, 61.0, 1.0], 'GSM4216506': [0.0, 58.0, 1.0], 'GSM4216507': [0.0, 79.0, 1.0], 'GSM4216508': [0.0, 69.0, 0.0], 'GSM4216509': [0.0, 57.0, 0.0], 'GSM4216510': [1.0, 46.0, 0.0], 'GSM4216511': [0.0, 44.0, 0.0], 'GSM4216512': [0.0, 46.0, 0.0], 'GSM4216513': [0.0, 63.0, 0.0], 'GSM4216514': [0.0, 59.0, 1.0], 'GSM4216515': [0.0, 81.0, 1.0], 'GSM4216516': [0.0, 60.0, 1.0], 'GSM4216517': [0.0, 92.0, 1.0], 'GSM4216518': [0.0, 45.0, 0.0], 'GSM4216519': [1.0, 47.0, 0.0], 'GSM4216520': [0.0, 27.0, 0.0], 'GSM4216521': [0.0, 58.0, 0.0], 'GSM4216522': [0.0, 57.0, 0.0], 'GSM4216523': [0.0, 38.0, 0.0], 'GSM4216524': [0.0, 45.0, 1.0], 'GSM4216525': [0.0, 51.0, 1.0], 'GSM4216526': [1.0, 70.0, 1.0], 'GSM4216527': [0.0, 57.0, 1.0], 'GSM4216528': [1.0, 56.0, 0.0], 'GSM421

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651278',
       'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651341', 'ILMN_1651343',
       'ILMN_1651347', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403',
       'ILMN_1651405', 'ILMN_1651429', 'ILMN_1651433', 'ILMN_1651438'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols in the gene annotation data
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Psoriatic_Arthritis/gene_data/GSE141934.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Psoriatic_Arthritis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE141934', './preprocessed/Psoriatic_Arthritis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Psoriatic_Arthritis/GSE141934.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Psoriatic_Arthritis', the least common label is '1.0' with 12 occurrences. This represents 12.00% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 49.75
  50% (Median): 56.0
  75%: 65.25
Min: 20.0
Max: 92.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 31 occurrences. This represents 31.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

